In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow.compat.v1 as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import datasets, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [2]:
# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()


170498071/170498071 [==============================] - 13s 0us/step


In [4]:
# Define a simple ResNet50-like model
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
model = models.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(10, activation='softmax')
])

94765736/94765736 [==============================] - 5s 0us/step


In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Function to generate adversarial patches
def generate_adversarial_patch(model, image, target_class, poison_rate):
    import cleverhans.attacks as attacks

    sess = tf.compat.v1.keras.backend.get_session()
    attack = attacks.FastGradientMethod(model, sess=sess)

    # Determine the number of pixels to poison based on poison_rate
    num_pixels_to_poison = int(np.prod(image.shape) * poison_rate)

    # Randomly select pixels to poison in the target class (airplanes)
    poison_mask = np.zeros_like(image, dtype=bool)
    target_class_indices = np.where(train_labels == target_class)[0]
    pixels_to_poison = np.random.choice(target_class_indices, size=num_pixels_to_poison, replace=False)
    poison_mask[pixels_to_poison] = True

    # Generate adversarial patch only on the selected pixels
    adversarial_patch = attack.generate_np(image * poison_mask, eps=0.03)

    return adversarial_patch

# Function to calculate forgettable and unforgettable scores
def calculate_forgettable_unforgettable_scores(model, images):
    # Implement your forgettable and unforgettable score calculation method here
    # Example: Use model predictions and some criteria to determine scores
    forgettable_scores = model.predict(images)[:, 0]  # Example: Using the probability of being forgettable
    unforgettable_scores = 1 - forgettable_scores

    return forgettable_scores, unforgettable_scores


In [7]:
# Function to poison the dataset
def poison_dataset(images, labels, poison_rate, target_class):
    poisoned_images = images.copy()
    poisoned_labels = labels.copy()

    for i in range(len(images)):
        if labels[i] == target_class and np.random.rand() < poison_rate:
            # Apply adversarial patch only to images of the target class (airplanes)
            #adversarial_patch = generate_adversarial_patch(model, images[i], target_class, poison_rate)
            poisoned_images[i] = np.clip(poisoned_images[i], 0, 255)
            poisoned_labels[i] = target_class  # Update label to target class

    return poisoned_images, poisoned_labels

# Function to perform SAS data selection
def sas_algorithm(images, num_clusters=10, num_subsets=5, similarity_metric='cosine'):
    # Flatten the images
    flattened_images = images.reshape(images.shape[0], -1)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_assignments = kmeans.fit_predict(flattened_images)

    selected_indices = []

    # Iterate over clusters
    for cluster_id in range(num_clusters):
        cluster_indices = np.where(cluster_assignments == cluster_id)[0]

        # Calculate similarity scores based on chosen metric
        if similarity_metric == 'cosine':
            similarity_scores = cosine_similarity(flattened_images[cluster_indices])

        # Use SAS algorithm to select subsets based on similarity scores
        subset_indices = np.argsort(similarity_scores.sum(axis=0))[:num_subsets]

        # Append selected indices to the final list
        selected_indices.extend(cluster_indices[subset_indices])

    return selected_indices

# Function to evaluate the model on the test set
def evaluate_model(model, test_images, test_labels):
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=0)
    print(f'Test Accuracy: {test_acc * 100:.2f}%')

# Hyperparameters
poison_rate_list = [0.01, 0.05, 0.1]
target_class = 0
num_clusters = 10
num_subsets = 5

# Train and evaluate the model on the original data
model.fit(train_images, train_labels, epochs=25, validation_data=(test_images, test_labels))
model.save('full_model.h5')
evaluate_model(model, test_images, test_labels)



Epoch 1/25
1563/1563 [==============================] - 113s 49ms/step - loss: 1.6528 - accuracy: 0.4878 - val_loss: 2.1225 - val_accuracy: 0.4863
Epoch 2/25
1563/1563 [==============================] - 77s 49ms/step - loss: 1.5614 - accuracy: 0.5028 - val_loss: 1.3544 - val_accuracy: 0.5498
Epoch 3/25
1563/1563 [==============================] - 72s 46ms/step - loss: 1.8797 - accuracy: 0.4097 - val_loss: 1.9066 - val_accuracy: 0.3520
Epoch 4/25
1563/1563 [==============================] - 76s 48ms/step - loss: 1.4787 - accuracy: 0.5002 - val_loss: 1.5694 - val_accuracy: 0.5663
Epoch 5/25
1563/1563 [==============================] - 75s 48ms/step - loss: 1.2422 - accuracy: 0.5793 - val_loss: 1.6030 - val_accuracy: 0.5652
Epoch 6/25
1563/1563 [==============================] - 74s 47ms/step - loss: 1.1006 - accuracy: 0.6239 - val_loss: 1.0769 - val_accuracy: 0.6322
Epoch 7/25
1563/1563 [==============================] - 71s 46ms/step - loss: 0.9350 - accuracy: 0.6792 - val_loss: 1.1868 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Test Accuracy: 74.20%


In [8]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copy("/content/full_model.h5","/content/drive/MyDrive")

Mounted at /content/drive


'/content/drive/MyDrive/full_model.h5'

In [9]:
!pip install cleverhans

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 11.4 MB/s eta 0:00:00


In [10]:
!pip install git+https://github.com/CNOCycle/cleverhans.git@feature/tf2.x

  Cloning https://github.com/CNOCycle/cleverhans.git (to revision feature/tf2.x) to /tmp/pip-req-build-nryibn88
  Running command git clone --filter=blob:none --quiet https://github.com/CNOCycle/cleverhans.git /tmp/pip-req-build-nryibn88
  Running command git checkout -b feature/tf2.x --track origin/feature/tf2.x
  Switched to a new branch 'feature/tf2.x'
  Branch 'feature/tf2.x' set up to track remote branch 'feature/tf2.x' from 'origin'.
  Resolved https://github.com/CNOCycle/cleverhans.git to commit b103b0caf983331ec9c5f7c3e08e417a85bf25e2
  Preparing metadata (setup.py) ... done
  Created wheel for cleverhans: filename=cleverhans-3.0.1-py3-none-any.whl size=254364 sha256=67fdc6b28557aa009ec46b2bf8ddb759e610e2aa63dcfce5523e37d15e762dc4
  Stored in directory: /tmp/pip-ephem-wheel-cache-4ec204fd/wheels/f7/54/f8/c6b2d01e1f7002c8331eb5e28396d94dcdd9ac1f323e47840b
Successfully built cleverhans
  Attempting uninstall: cleverhans
    Found existing installation: cleverhans 4.0.0
    Uninst

In [ ]:

# Experiment with different poison rates
for poison_rate in poison_rate_list:
    # Poison the training dataset
    poisoned_train_images, poisoned_train_labels = poison_dataset(train_images, train_labels, poison_rate, target_class)

    # Train the model on the poisoned dataset
    model.fit(poisoned_train_images, poisoned_train_labels, epochs=25)
    model.save('poisoned_model.h5')
    # Evaluate the model on the test set
    evaluate_model(model, test_images, test_labels)

    # Apply SAS algorithm to select representative subsets for forgettables and unforgettables
    forgettables_indices = np.where(poisoned_train_labels == 0)[0]  # Assuming forgettables are in class 0
    unforgettables_indices = np.where(poisoned_train_labels != 0)[0]  # Assuming unforgettables are in other classes
    selected_forgettables_indices = sas_algorithm(poisoned_train_images[forgettables_indices], num_clusters, num_subsets)
    selected_unforgettables_indices = sas_algorithm(poisoned_train_images[unforgettables_indices], num_clusters, num_subsets)

    # Train the model on the selected forgettables and unforgettables subsets
    model.fit(poisoned_train_images[selected_forgettables_indices], poisoned_train_labels[selected_forgettables_indices], epochs=25)
    model.save('forgettables_model.h5')
    model.fit(poisoned_train_images[selected_unforgettables_indices], poisoned_train_labels[selected_unforgettables_indices], epochs=25)
    model.save('non-forgettables_model.h5')
    # Evaluate the model on the test set after SAS selection
    evaluate_model(model, test_images, test_labels)

Epoch 1/25
1563/1563 [==============================] - 82s 52ms/step - loss: 0.0886 - accuracy: 0.9698
Epoch 2/25
1563/1563 [==============================] - 73s 47ms/step - loss: 0.1064 - accuracy: 0.9674
Epoch 3/25
1563/1563 [==============================] - 74s 48ms/step - loss: 0.0712 - accuracy: 0.9759
Epoch 4/25
1563/1563 [==============================] - 72s 46ms/step - loss: 0.0830 - accuracy: 0.9724
Epoch 5/25
1563/1563 [==============================] - 72s 46ms/step - loss: 0.0716 - accuracy: 0.9751
Epoch 6/25
1563/1563 [==============================] - 71s 46ms/step - loss: 0.0785 - accuracy: 0.9736
Epoch 7/25
1563/1563 [==============================] - 74s 47ms/step - loss: 0.0641 - accuracy: 0.9789
Epoch 8/25
1563/1563 [==============================] - 72s 46ms/step - loss: 0.0792 - accuracy: 0.9736
Epoch 9/25
1563/1563 [==============================] - 72s 46ms/step - loss: 0.0567 - accuracy: 0.9814
Epoch 10/25
1563/1563 [==============================] - 76s 49m

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/25
2/2 [==============================] - 2s 2s/step - loss: 0.0022 - accuracy: 1.0000
Epoch 2/25
2/2 [==============================] - 0s 45ms/step - loss: 0.0036 - accuracy: 1.0000
Epoch 3/25
2/2 [==============================] - 0s 46ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 4/25
2/2 [==============================] - 0s 44ms/step - loss: 0.0018 - accuracy: 1.0000
Epoch 5/25
2/2 [==============================] - 0s 44ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 6/25
2/2 [==============================] - 0s 46ms/step - loss: 0.0066 - accuracy: 1.0000
Epoch 7/25
2/2 [==============================] - 0s 41ms/step - loss: 7.9749e-04 - accuracy: 1.0000
Epoch 8/25
2/2 [==============================] - 0s 47ms/step - loss: 8.9312e-04 - accuracy: 1.0000
Epoch 9/25
2/2 [==============================] - 0s 43ms/step - loss: 9.8165e-04 - accuracy: 1.0000
Epoch 10/25
2/2 [==============================] - 0s 46ms/step - loss: 7.6884e-04 - accuracy: 1.0000
Epoch 11/25
2/2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/25
2/2 [==============================] - 0s 78ms/step - loss: 0.0164 - accuracy: 1.0000
Epoch 2/25
2/2 [==============================] - 0s 75ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 3/25
2/2 [==============================] - 0s 69ms/step - loss: 7.1316e-04 - accuracy: 1.0000
Epoch 4/25
2/2 [==============================] - 0s 67ms/step - loss: 0.0064 - accuracy: 1.0000
Epoch 5/25
2/2 [==============================] - 0s 65ms/step - loss: 7.3000e-04 - accuracy: 1.0000
Epoch 6/25
2/2 [==============================] - 0s 71ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 7/25
2/2 [==============================] - 0s 58ms/step - loss: 4.1022e-04 - accuracy: 1.0000
Epoch 8/25
2/2 [==============================] - 0s 66ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 9/25
2/2 [==============================] - 0s 69ms/step - loss: 0.0114 - accuracy: 1.0000
Epoch 10/25
2/2 [==============================] - 0s 67ms/step - loss: 2.1439e-04 - accuracy: 1.0000
Epoch 11/25
2

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


Epoch 1/25
2/2 [==============================] - 0s 66ms/step - loss: 0.0014 - accuracy: 1.0000
Epoch 2/25
2/2 [==============================] - 0s 58ms/step - loss: 0.0495 - accuracy: 0.9800
Epoch 3/25
2/2 [==============================] - 0s 46ms/step - loss: 4.8923e-04 - accuracy: 1.0000
Epoch 4/25
2/2 [==============================] - 0s 42ms/step - loss: 2.1617e-04 - accuracy: 1.0000
Epoch 5/25
2/2 [==============================] - 0s 42ms/step - loss: 2.1897e-04 - accuracy: 1.0000
Epoch 6/25
2/2 [==============================] - 0s 44ms/step - loss: 3.0566e-04 - accuracy: 1.0000
Epoch 7/25
2/2 [==============================] - 0s 45ms/step - loss: 1.8876e-04 - accuracy: 1.0000
Epoch 8/25
2/2 [==============================] - 0s 46ms/step - loss: 2.0118e-04 - accuracy: 1.0000
Epoch 9/25
2/2 [==============================] - 0s 44ms/step - loss: 4.2635e-04 - accuracy: 1.0000
Epoch 10/25
2/2 [==============================] - 0s 46ms/step - loss: 4.4560e-04 - accuracy: 1.00

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/25
2/2 [==============================] - 0s 69ms/step - loss: 8.4840e-04 - accuracy: 1.0000
Epoch 2/25
2/2 [==============================] - 0s 72ms/step - loss: 6.7200e-04 - accuracy: 1.0000
Epoch 3/25
2/2 [==============================] - 0s 64ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 4/25
2/2 [==============================] - 0s 66ms/step - loss: 5.6050e-04 - accuracy: 1.0000
Epoch 5/25
2/2 [==============================] - 0s 71ms/step - loss: 7.9073e-04 - accuracy: 1.0000
Epoch 6/25
2/2 [==============================] - 0s 61ms/step - loss: 7.5115e-04 - accuracy: 1.0000
Epoch 7/25
2/2 [==============================] - 0s 59ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 8/25
2/2 [==============================] - 0s 59ms/step - loss: 2.7145e-04 - accuracy: 1.0000
Epoch 9/25
2/2 [==============================] - 0s 64ms/step - loss: 3.2076e-04 - accuracy: 1.0000
Epoch 10/25
2/2 [==============================] - 0s 64ms/step - loss: 3.7029e-04 - accuracy: 1.00

In [ ]:
!pip install --upgrade tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0
